# Model generator

## Imports

In [3]:
# list of supported layers
from tensorflow.keras.layers import (
    Input,
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
    Lambda, # only for polynomial activation in the form of `Lambda(lambda x: x**2+x)`
    MaxPooling2D,
    ReLU,
    Softmax,
    )
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import tf2onnx
import onnxruntime as ort

## Dataset generator

In [4]:
X_train = np.random.randint(0, 255, size=(50, 32, 32, 3))
X_test = np.random.randint(0, 255, size=(10, 32, 32, 3))

y_train = np.random.randint(0, 4, size=(50, 1))
y_test = np.random.randint(0, 4, size=(10, 1))

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50, 32, 32, 3), (10, 32, 32, 3), (50, 4), (10, 4))

## Model definition

### Formulation

In [69]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=30, kernel_size=2, strides=1, use_bias = True,input_shape=(32, 32, 3) ), 
    tf.keras.layers.MaxPooling2D(pool_size=(1, 1), padding='valid'),
    tf.keras.layers.Conv2D(filters=10, kernel_size=2, strides=2, use_bias = False),
    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides = (1,1), padding='valid'),
    #tf.keras.layers.GlobalMaxPool2D(),
    #tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1500, activation='relu'),
    tf.keras.layers.Dense(1200, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax'),
])


In [70]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 31, 31, 30)        390       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 31, 30)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 15, 15, 10)        1200      
                                                                 
 dense_10 (Dense)            (None, 15, 15, 8)         88        
                                                                 
 batch_normalization_4 (Batc  (None, 15, 15, 8)        32        
 hNormalization)                                                 
                                                                 
 dense_11 (Dense)            (None, 15, 15, 12)       

In [71]:
num = 2
model.layers[num].name, len(model.layers[num].weights), model.layers[num].weights

('conv2d_6',
 1,
 [<tf.Variable 'conv2d_6/kernel:0' shape=(2, 2, 30, 10) dtype=float32, numpy=
  array([[[[ 0.151755  ,  0.02965891,  0.11027792, ..., -0.07940943,
             0.0450602 , -0.06026942],
           [ 0.06969878,  0.17508283, -0.19216797, ..., -0.18597053,
            -0.16191572,  0.08852878],
           [ 0.14598835,  0.08625799, -0.04200442, ...,  0.07254735,
             0.11810941,  0.09656203],
           ...,
           [-0.18034564, -0.17960891, -0.17029476, ..., -0.18624519,
            -0.14101855, -0.13976574],
           [ 0.1872369 ,  0.13522726,  0.16895595, ..., -0.15175575,
             0.02538882, -0.02522728],
           [-0.00817746,  0.03266709, -0.08084045, ...,  0.04074501,
             0.13202065, -0.18967809]],
  
          [[ 0.0248781 , -0.13570522,  0.11865121, ..., -0.06770051,
             0.17224339, -0.07759344],
           [-0.13546255, -0.1461629 ,  0.0968166 , ..., -0.13745408,
             0.1389429 ,  0.14482135],
           [ 0.128182

### Training

In [72]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [73]:
model.fit(X_train, y_train, epochs=15, batch_size=16,  validation_data=(X_test, y_test))

Epoch 1/15


4/4 [==============================] - 1s 136ms/step - loss: 1.8046 - acc: 0.3000 - val_loss: 8.7168 - val_acc: 0.0000e+00
Epoch 2/15
4/4 [==============================] - 0s 32ms/step - loss: 1.3488 - acc: 0.4000 - val_loss: 6.9429 - val_acc: 0.0000e+00
Epoch 3/15
4/4 [==============================] - 0s 58ms/step - loss: 1.1548 - acc: 0.4600 - val_loss: 6.6057 - val_acc: 0.0000e+00
Epoch 4/15
4/4 [==============================] - 0s 30ms/step - loss: 1.0575 - acc: 0.6200 - val_loss: 5.9271 - val_acc: 0.0000e+00
Epoch 5/15
4/4 [==============================] - 0s 43ms/step - loss: 0.9802 - acc: 0.7400 - val_loss: 5.0733 - val_acc: 0.0000e+00
Epoch 6/15
4/4 [==============================] - 0s 43ms/step - loss: 0.9042 - acc: 0.7400 - val_loss: 4.4517 - val_acc: 0.0000e+00
Epoch 7/15
4/4 [==============================] - 0s 40ms/step - loss: 0.8396 - acc: 0.8000 - val_loss: 4.0562 - val_acc: 0.0000e+00
Epoch 8/15
4/4 [==============================] - 0s 40ms/step - loss: 0.7777 -

## Save

### Tensorflow model - h5

In [84]:
# Save tensorflow model
model.save('model_dense.h5')

In [75]:
model.layers

In [76]:
model.load_weights('model_dense.h5')

In [77]:
num = 1
model.layers[num].name, len(model.layers[num].weights), model.layers[num].weights

('max_pooling2d_1', 0, [])

In [78]:
for layer in model.layers:
    print(layer.__class__.__name__, layer.get_config())
    try:
        print(layer.get_config()['function'])
    except:
        pass
    print(layer.get_input_shape_at(0),layer.get_output_shape_at(0))
    print("output:", layer.output)
    try:
        print(layer.get_weights()[0].shape)
        print(layer.get_weights()[1].shape)
    except:
        pass

Conv2D {'name': 'conv2d_5', 'trainable': True, 'batch_input_shape': (None, 32, 32, 3), 'dtype': 'float32', 'filters': 30, 'kernel_size': (2, 2), 'strides': (1, 1), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
(None, 32, 32, 3) (None, 31, 31, 30)
output: KerasTensor(type_spec=TensorSpec(shape=(None, 31, 31, 30), dtype=tf.float32, name=None), name='conv2d_5/BiasAdd:0', description="created by layer 'conv2d_5'")
(2, 2, 3, 30)
(30,)
MaxPooling2D {'name': 'max_pooling2d_1', 'trainable': True, 'dtype': 'float32', 'pool_size': (1, 1), 'padding': 'valid', 'strides': (1, 1), 'data_format': 'channels_last'}
(None, 31, 31, 30) (None, 31, 31

### Onnx model - onnx

In [79]:
#Convert to Onnx
# Define the input signature for the conversion
input_signature = [tf.TensorSpec([None, 32, 32, 3], tf.float32, name="input")]

# Convert the model to ONNX
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, opset=13)

# Save the ONNX model to a file
with open("model_dense.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [80]:
for node in onnx_model.graph.node:
    if 'global' in node.name:
        print(node.attribute)

## Input Json generator

In [81]:
import json


In [82]:
X = X_test[[0]]
y = model.predict(X)
y

1/1 [==============================] - 1s 577ms/step


array([[0.03742743, 0.06317029, 0.89477825, 0.00462401]], dtype=float32)

In [83]:
with open("input.json", "w") as f:
    json.dump({'in': X.flatten().tolist()}, f)